# Operations

`DataFrame` and `Series` objects allow for several operations to take place either on a single object, or between two or more objects.

For this section, we will import a complementary dataset of transits (voyages) of each vessel. This is a much larger dataset, since ships each have multiple transits over several years.

For example, we can perform arithmetic on the elements of two objects, such as combining distances travelled across years. First, let's artificially construct two Series, consisting of segment lengths travelled in 2009 and 2010:

In [ ]:
import pandas as pd

segments = pd.read_csv('../data/AIS/transit_segments.csv', parse_dates=['st_time'])

In [ ]:
segments.shape

Extract the year from the date (more on dates/times later!)

In [ ]:
segments['year'] = segments.st_time.dt.year
segments.year.head()

In [ ]:
segments2009 = segments[segments.year==2009].copy()
lengths2009 = pd.Series({name: segments2009[segments2009.name==name].seg_length.sum() 
                         for name in segments2009.name.unique()})

In [ ]:
segments2010 = segments[segments.year==2010].copy()
lengths2010 = pd.Series({name: segments2010[segments2010.name==name].seg_length.sum() 
                         for name in segments2010.name.unique()})

We will learn a much more efficient way of performing this operation when we get to `groupby` operations a little later on.

In [ ]:
length2009 = segments.loc[segments.year==2009, 'seg_length']
length2009.index = segments.name[segments.year==2009]

length2010 = segments.loc[segments.year==2010, 'seg_length']
length2010.index = segments.name[segments.year==2010]

Now, let's add them together, in hopes of getting 2-year travel totals:

In [ ]:
seg_lengths = lengths2009 + lengths2010
seg_lengths

Pandas' data alignment places `NaN` values for labels that do not overlap in the two Series. In fact, the majority of ships only have data for one of the two years.

In [ ]:
seg_lengths.notnull().mean()

While we do want the operation to honor the data labels in this way, we probably do not want the missing values to be filled with `NaN`. We can use the `add` method to calculate ship travel totals by using the `fill_value` argument to insert a zero for years that do not have data for a particular vessel:

In [ ]:
lengths2009.add(lengths2010, fill_value=0)

Operations can also be **broadcast** between rows or columns.

For example, if we subtract the maximum LOA (ship length) from the `max_loa` column, we get a the differences between the size of a given vessel to the largest vessel.

In [ ]:
vessels = pd.read_csv('../data/AIS/vessel_information.csv', index_col=0)

vessels.max_loa - vessels.max_loa.max()

Or, looking at things row-wise, we can see how a particular vessel compares with the rest of the group with respect to important features:

In [ ]:
vessels[vessels.max_loa==vessels.max_loa.max()]

In [ ]:
recs = vessels[['num_names','num_loas', 'max_loa', 'num_types']]
diff = recs - recs.loc[354092000]
diff[:10]

We can also apply functions to each column or row of a `DataFrame`

In [ ]:
import numpy as np

recs.apply(np.median)

In [ ]:
def range_calc(x):
    return x.max() - x.min()

In [ ]:
recs.apply(range_calc)

## Sorting and Ranking

Pandas objects include methods for re-ordering data.

In [ ]:
segments.sort_index().head()

In [ ]:
segments.sort_index(ascending=False).head()

Try sorting the **columns** instead of the rows, in ascending order:

In [ ]:
segments.sort_index(axis=1).head()

We can also use `sort_values` to sort a `Series` by value, rather than by label.

In [ ]:
segments.seg_length.sort_values(ascending=False).head(10)

For a `DataFrame`, we can sort according to the values of one or more columns using the `by` argument of `sort_values`:

In [ ]:
segments[['avg_sog','max_sog','min_sog']].sort_values(ascending=[False,True], 
                                           by=['max_sog', 'min_sog']).head(10)

A related method `rank` does not re-arrange data, but instead returns an index that ranks each value relative to others in the Series.

In [ ]:
segments.avg_sog.rank()

Ties are assigned the mean value of the tied ranks, which may result in decimal values.

In [ ]:
pd.Series([100,100,100]).rank()

Alternatively, you can break ties via one of several methods, such as by the order in which they occur in the dataset:

In [ ]:
segments.avg_sog.rank(method='first').sort_values()

Calling the `DataFrame`'s `rank` method results in the ranks of all columns:

In [ ]:
segments.rank(ascending=False).head()

## Hierarchical indexing

While the `mmsi` (Maritime Mobile Service Identity) field represents a unique identifier in the `vessels` table, it does not in the `segments` table.

We can use hierarchical indexing to build a **unique index** that may be more meaningful than the ordered set of integers that are given as a default index.

In [ ]:
segments_hier = segments.set_index(['mmsi', 'name', 'year', 'transit', 'segment']).sort_index(ascending=True)
segments_hier.head()

This index is a `MultiIndex` object that consists of a sequence of tuples, the elements of which is some combination of the three columns used to create the index. Where there are multiple repeated values, Pandas does not print the repeats, making it easy to identify groups of values.

In [ ]:
segments_hier.index.is_unique

Try using this hierarchical index to retrieve the first segment of the tenth transit of the *Sentinel* (mmsi=366766980):

In [ ]:
segments_hier.loc[(366766980, 'Sentinel', 2009, 10, 1)]

With a hierachical index, we can select subsets of the data based on a *partial* index:

In [ ]:
segments_hier.loc[9]

Hierarchical indices can be created on either or both axes. Here is a trivial example:

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape(( 4, 3)), 
                  index =[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], 
                  columns =[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

frame

If you want to get fancy, both the row and column indices themselves can be given names:

In [ ]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

With this, we can do all sorts of custom indexing:

In [ ]:
frame.loc['a', 'Ohio']

Additionally, the order of the set of indices in a hierarchical `MultiIndex` can be changed by swapping them pairwise:

In [ ]:
segments_hier.swaplevel('mmsi', 'name').head()

In [ ]:
segments_hier.swaplevel('mmsi', 'name').loc['Sentinel'].head()

Perhaps the easiest way to extract data from arbitrary levels of a `MultiIndex` is to use the `IndexSlice` function. For example, if we want the first two transits from all trips in 2009 and 2010:

In [ ]:
segments_hier.loc[pd.IndexSlice[:, :, 2009:2010, [1,2]], 'seg_length'].head(12)

The `:` tells pandas to include the entire level. What `IndexSlice` does is generate a Python `slice` object that represents what is required from each level. So, the above is equivalent to:

    (slice(None, None, None), slice(None, None, None), [2009, 2010], [1, 2])

Data can also be sorted by any index level, using `sort_index` with the `level` argument appropriately specified:

In [ ]:
segments_hier.sort_index(level='transit', ascending=True).head()

## Missing data

The occurence of missing data is so prevalent that it pays to use tools like Pandas, which seamlessly integrates missing data handling so that it can be dealt with easily, and in the manner required by the analysis at hand.

Missing data are represented in `Series` and `DataFrame` objects by the `NaN` floating point value. However, `None` is also treated as missing, since it is commonly used as such in other contexts (*e.g.* NumPy).

In [ ]:
foo = pd.Series([np.nan, -3, None, 'foobar', np.inf])
foo

In [ ]:
foo.isnull()

To illustrate how to deal with missing values in pandas, we will import an educational database, which reports test scores for children with hearing disabilities, along with associated covariates, several of which are missing.

In [ ]:
testing = pd.read_csv('../data/test_scores.csv', index_col=0)
testing.head(10)

Missing values may be dropped or indexed out:

By default, `dropna` drops entire rows in which one or more values are missing.

In [ ]:
testing.dropna().head(10)

This can be overridden by passing the `how='all'` argument, which only drops a row when every field is a missing value.

In [ ]:
testing.dropna(how='all')

This can be customized further by specifying how many values need to be present before a row is dropped via the `thresh` argument.

In [ ]:
testing.dropna(thresh=10)

This is typically used in time series applications, where there are repeated measurements that are incomplete for some subjects.

Rather than omitting missing data from an analysis, in some cases it may be suitable to fill the missing value in, either with a default value (such as zero) or a value that is either imputed or carried forward/backward from similar data points. We can do this programmatically in Pandas with the `fillna` argument.

In [ ]:
testing.fillna(0).head(10)

In [ ]:
testing.fillna({'family_inv': 0, 'mother_hs':1}).head(10)

Notice that `fillna` by default returns a new object with the desired filling behavior, rather than changing the `Series` or  `DataFrame` in place (**in general, we like to do this, by the way!**).

Missing values can also be interpolated, using any one of a variety of methods:

In [ ]:
testing.fillna(method='bfill').head(10)

## Data summarization

We often wish to summarize data in `Series` or `DataFrame` objects, so that they can more easily be understood or compared with similar data. The NumPy package contains several functions that are useful here, but several summarization or reduction methods are built into Pandas data structures.

In [ ]:
testing.sum()

Clearly, `sum` is more meaningful for some columns than others. For methods like `mean` for which application to string variables is not just meaningless, but impossible, these columns are automatically exculded:

In [ ]:
testing.mean()

The important difference between NumPy's functions and Pandas' methods is that the latter have built-in support for handling missing data.

Sometimes we may not want to ignore missing values, and allow the `nan` to propagate.

In [ ]:
testing.mean(skipna=False)

Passing `axis=1` will summarize over rows instead of columns, which only makes sense in certain situations.

In [ ]:
nonenglish_nonwhite_withHS = testing[['non_english', 'mother_hs', 'non_white']].prod(axis=1)

nonenglish_nonwhite_withHS

A useful summarization that gives a quick snapshot of multiple statistics for a `Series` or `DataFrame` is `describe`:

In [ ]:
testing.describe()

`describe` can detect non-numeric data and sometimes yield useful information about it.

In [ ]:
testing.non_english.describe()

We can also calculate summary statistics *across* multiple columns, for example, correlation and covariance.

$$cov(x,y) = \sum_i (x_i - \bar{x})(y_i - \bar{y})$$

In [ ]:
testing.score.cov(testing.age_test)

$$corr(x,y) = \frac{cov(x,y)}{(n-1)s_x s_y} = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i (x_i - \bar{x})^2 \sum_i (y_i - \bar{y})^2}}$$

In [ ]:
testing.score.corr(testing.age_test)

If we have a `DataFrame` with a hierarchical index (or indices), summary statistics can be applied with respect to any of the index levels:

In [ ]:
segments_hier.head()

In [ ]:
segments_hier.sum(level='transit').seg_length

## Writing Data to Files

As well as being able to read several data input formats, Pandas can also export data to a variety of storage formats. We will bring your attention to just a couple of these.

In [ ]:
testing.to_csv("testing.csv")

In [ ]:
!cat testing.csv

The `to_csv` method writes a `DataFrame` to a comma-separated values (csv) file. You can specify custom delimiters (via `sep` argument), how missing values are written (via `na_rep` argument), whether the index is writen (via `index` argument), whether the header is included (via `header` argument), among other options.